In [1]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from typing import List
import os

Data Models: Structured insights we from from discovery interview transcripts

In [17]:
class PainPoint(BaseModel):
    description: str = Field(description="What's not working for the user")
    impact: str = Field(description="Time lost, extra work, costs, uncertainty, etc.")
    quote: str = Field(description="User's exact words")

class InterviewInsights(BaseModel):
    """All insights extracted from one interview"""
    pain_points: List[PainPoint] = Field(description="All pain points mentioned in the interview")

class JobToBeDone(BaseModel):
    functional_job: str = Field(description="What task are they trying to complete?")
    emotional_job: str = Field(description="What they care about and how do they want to feel?")
    context: str = Field(description="When/where does this happen?")
    quote: str

class Workaround(BaseModel):
    what_they_do: str = Field(description="The workaround(s) they've created")
    why_needed: str = Field(description="What problem does the workaround solve?")
    cost: str = Field(description="Time/effort/risks this workaround takes")
    quote: str

class DesiredOutcome(BaseModel):
    outcome: str = Field(description="What do they really want?")
    current_gap: str = Field(description="Why can't they achieve this now?")
    quote: str

class BehavioralSignal(BaseModel):
    observation: str = Field(description="What implicit things did they say/do that was revealing?")
    what_it_reveals: str = Field(description="The underlying need or belief")
    quote: str

Create an AI agent that extracts insights

In [18]:
insight_agent = Agent(
    model='openai:gpt-4o-mini',
    output_type=InterviewInsights,  # ← Returns multiple pain points
    system_prompt="""You are an expert product researcher analyzing user interviews.  
    Extract ALL pain points from the interview transcript.
    Focus on problems that cost the user time, money, create uncertainty, or cause frustration.
    A single interview may have multiple pain points - find them all.
    Always include the user's exact words as evidence for each pain point."""
)

Sample Interview:

In [19]:
SAMPLE_INTERVIEW = """
Interviewer: Tell me about the last time you tried to analyze user research data.

Participant: Oh, just last week. We had 12 customer interviews and I needed to present insights to the team. I probably spent 8 hours just going through transcripts, copying quotes into a doc, trying to find themes.

Interviewer: What made it take so long?

Participant: The biggest problem is that insights are scattered everywhere. One user mentions pricing is confusing in passing, another user describes the same issue but uses completely different words. I have to read everything multiple times and manually connect the dots. By the end, my brain is fried and I'm not even confident I caught all the important patterns.

Interviewer: How does that affect your work?

Participant: It makes me anxious honestly. When my PM asks "what did users say about feature X?" I sometimes have to say "let me go back and check" because I can't remember or didn't catch it the first time. It makes me look unprepared even though I've put in the hours.
"""

Extract pain points from the interview

In [22]:
result = await insight_agent.run(SAMPLE_INTERVIEW)
insights = result.output

print("=" * 60)
print(f"EXTRACTED {len(insights.pain_points)} PAIN POINTS")
print("=" * 60)

for i, pain_point in enumerate(insights.pain_points, 1):
    print(f"\n--- Pain Point {i} ---")
    print(f"Description: {pain_point.description}")
    print(f"Impact: {pain_point.impact}")
    print(f"Quote: \"{pain_point.quote}\"")

print("\n" + "=" * 60)

EXTRACTED 5 PAIN POINTS

--- Pain Point 1 ---
Description: Insights are scattered everywhere, making it difficult to connect them.
Impact: Costing time through multiple readings and manual connection of dots.
Quote: "The biggest problem is that insights are scattered everywhere."

--- Pain Point 2 ---
Description: Users mention similar issues using completely different words, complicating analysis.
Impact: Creating uncertainty and increasing time needed to find relevant patterns.
Quote: "another user describes the same issue but uses completely different words."

--- Pain Point 3 ---
Description: Participant's brain is 'fried' after extended analysis, leading to frustration.
Impact: Causing mental fatigue and reducing confidence in insights.
Quote: "By the end, my brain is fried."

--- Pain Point 4 ---
Description: Anxiety caused by not being able to quickly recall user feedback during discussions.
Impact: Making the participant feel unprepared and frustrated.
Quote: "It makes me anxio